In [5]:
import pandas as pd
import os


In [1]:
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

In [6]:
# Data  Extraction

try:
    data = pd.read_csv(r'rawdata/zipco_transaction.csv')
    print("Data extracted successfully.")
except Exception as e:
    print(f"Error reading CSV file: {e}")

Data extracted successfully.


In [7]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

Data Cleaning and Transformation

In [9]:
# Remove duplicate entries
data.drop_duplicates(inplace=True)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             900 non-null    float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        900 non-null    object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [11]:
# Handling missing values

numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    data[col].fillna(data[col].mean(), inplace=True)

/var/folders/7f/4mz0btqn1ybc1zkv_953606m0000gn/T/ipykernel_28914/2454251396.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        900 non-null    object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [14]:
# Handling missing values

string_cols = data.select_dtypes(include=['object']).columns
for col in string_cols:
    data[col].fillna(('unknown'), inplace=True)

/var/folders/7f/4mz0btqn1ybc1zkv_953606m0000gn/T/ipykernel_28914/718332365.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(('unknown'), inplace=True)


In [34]:
# Change date column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [16]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [20]:
# Product table

product = data[['ProductName', 'UnitPrice']].drop_duplicates().reset_index(drop=True)
product.index.name = 'ProductID'
product = product.reset_index()


In [21]:
product.head()

,ProductID,ProductName,UnitPrice
0,0,Vanilla Cake,12.532304
1,1,Red Velvet Cake,7.083070
2,2,Chocolate Cake,6.736064
3,3,Carrot Cake,7.314823
4,4,Pizza Pepperoni,7.577727


In [22]:
# Customer table

customer = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customer.index.name = 'CustomerID'
customer = customer.reset_index()                


In [23]:
customer.head()

,CustomerID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net


In [24]:
# Staff table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff = staff.reset_index()


In [25]:
staff.head()

,StaffID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org


In [26]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [36]:
# Transactions table
transaction = data.merge(product, on=['ProductName', 'UnitPrice'], how='left') \
    .merge(customer, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how='left') \
    .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left') \

transaction.index.name = 'TransactionID'
transaction = transaction.reset_index() \
                            [['Date', 'TransactionID', 'ProductID', 'UnitPrice', 'Quantity', 'StoreLocation', 'PaymentType', \
                                'PromotionApplied', 'Weather', 'Temperature', 'StaffPerformanceRating', 'CustomerFeedback', \
                                'DeliveryTime_min', 'OrderType', 'CustomerID', 'StaffID', 'DayOfWeek', 'TotalSales']]

In [31]:
transaction.head()

,Date,TransactionID,ProductID,UnitPrice,Quantity,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,CustomerFeedback,DeliveryTime_min,OrderType,CustomerID,StaffID,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,0,0,12.532304,2,South,Cash,True,Rainy,20.654914,Poor,Neutral,30,In Store,0,0,Sunday,25.064608
1,2023-01-01 01:00:00,1,1,7.083070,1,South,Cash,False,Rainy,23.549497,Average,Positive,33,In Store,1,1,Sunday,7.083070
2,2023-01-01 02:00:00,2,2,6.736064,5,North,Cash,True,Rainy,27.154342,Excellent,unknown,43,Phone Order,2,2,Sunday,33.680321
3,2023-01-01 03:00:00,3,3,7.314823,2,North,Cash,False,Cloudy,20.137483,Poor,Positive,32,Online Order,3,3,Sunday,14.629647
4,2023-01-01 04:00:00,4,4,7.577727,1,East,Credit Card,True,Cloudy,23.020987,Good,Neutral,58,In Store,4,4,Sunday,7.577727


In [32]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   TransactionID           1000 non-null   int64  
 2   ProductID               1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   Quantity                1000 non-null   int64  
 5   StoreLocation           1000 non-null   object 
 6   PaymentType             1000 non-null   object 
 7   PromotionApplied        1000 non-null   bool   
 8   Weather                 1000 non-null   object 
 9   Temperature             1000 non-null   float64
 10  StaffPerformanceRating  1000 non-null   object 
 11  CustomerFeedback        1000 non-null   object 
 12  DeliveryTime_min        1000 non-null   int64  
 13  OrderType               1000 non-null   object 
 14  CustomerID              1000 non-null   i

In [37]:
# Save the tables as CSV files
data.to_csv("cleandata/cleandata.csv", index=False)
product.to_csv("cleandata/product.csv", index=False)
customer.to_csv("cleandata/customer.csv", index=False)
staff.to_csv("cleandata/staff.csv", index=False)
transaction.to_csv("cleandata/transaction.csv", index=False)